# Process Google Timeline Data

In [20]:
import os
from dotenv import load_dotenv

# VARIABLES
folder = "EZ_data" # ONLY need to change this value

json_file = folder + "/location-history.json"
cluster_counts_file_path = folder + "/cluster_counts.csv"
cluster_data_file_path = folder + "/cluster_means.csv"
places_data_file_path = folder + "/places_info.csv"

load_dotenv()
# Make sure to create a .env file and store your API KEY
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [21]:
import json
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from datetime import datetime
import matplotlib.pyplot as plt

### Load Data

In [22]:
def load_google_timeline(json_file):
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)
    
    locations = []
    for entry in data:
        if "visit" in entry:
            visit_info = entry["visit"]
            top_candidate = visit_info.get("topCandidate", {})

            if "placeLocation" in top_candidate:
                latlon_str = top_candidate["placeLocation"].replace("geo:", "")
                lat, lon = map(float, latlon_str.split(",")) 
                
                start_time = entry.get("startTime", "")
                end_time = entry.get("endTime", "")

                locations.append((lat, lon, start_time, end_time))

    return pd.DataFrame(locations, columns=["latitude", "longitude", "start_time", "end_time"])


df = load_google_timeline(json_file)
df.head()

,latitude,longitude,start_time,end_time
0,38.028028,-78.508516,2025-02-12T17:56:54.000-05:00,2025-02-13T10:44:48.307-05:00
1,38.031619,-78.510846,2025-02-13T10:51:39.463-05:00,2025-02-13T12:31:04.514-05:00
2,38.031619,-78.510846,2025-02-13T10:51:39.463-05:00,2025-02-13T11:03:44.537-05:00
3,38.028028,-78.508516,2025-02-13T12:36:02.657-05:00,2025-02-13T16:04:27.150-05:00
4,38.052815,-78.501257,2025-02-13T16:15:59.999-05:00,2025-02-13T17:02:51.018-05:00


In [23]:
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])
df['duration_time'] = df['end_time'] - df['start_time']
df.head()

C:\Users\chery\AppData\Local\Temp\ipykernel_38096\2510686417.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['start_time'] = pd.to_datetime(df['start_time'])
C:\Users\chery\AppData\Local\Temp\ipykernel_38096\2510686417.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['end_time'] = pd.to_datetime(df['end_time'])


,latitude,longitude,start_time,end_time,duration_time
0,38.028028,-78.508516,2025-02-12 17:56:54-05:00,2025-02-13 10:44:48.307000-05:00,0 days 16:47:54.307000
1,38.031619,-78.510846,2025-02-13 10:51:39.463000-05:00,2025-02-13 12:31:04.514000-05:00,0 days 01:39:25.051000
2,38.031619,-78.510846,2025-02-13 10:51:39.463000-05:00,2025-02-13 11:03:44.537000-05:00,0 days 00:12:05.074000
3,38.028028,-78.508516,2025-02-13 12:36:02.657000-05:00,2025-02-13 16:04:27.150000-05:00,0 days 03:28:24.493000
4,38.052815,-78.501257,2025-02-13 16:15:59.999000-05:00,2025-02-13 17:02:51.018000-05:00,0 days 00:46:51.019000


### Find Clusters based on Longitude and Latitude

In [ ]:
def cluster_locations(df, eps=50, min_samples=2):
    coords = df[["latitude", "longitude"]].values
    db = DBSCAN(eps=eps / 6371000, min_samples=min_samples, metric="haversine").fit(np.radians(coords))
    
    df["cluster"] = db.labels_
    return df

df_clusters = cluster_locations(df)
df_clusters["duration_time"] = pd.to_timedelta(df_clusters["duration_time"])

In [25]:
df_clusters.to_csv(cluster_counts_file_path)

In [26]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

silhouette = silhouette_score(df[["latitude", "longitude"]], df_clusters['cluster'])
db_index = davies_bouldin_score(df[["latitude", "longitude"]], df_clusters['cluster'])
ch_index = calinski_harabasz_score(df[["latitude", "longitude"]], df_clusters['cluster'])

print(f"Silhouette Score: {silhouette:.2f}")
print(f"Davies-Bouldin Index: {db_index:.2f}")
print(f"Calinski-Harabasz Index: {ch_index:.2f}")

Silhouette Score: 0.45
Davies-Bouldin Index: 1.06
Calinski-Harabasz Index: 6.57


In [27]:
import folium

def plot_clusters_folium(df):
    # map_center = [df["latitude"].mean(), df["longitude"].mean()]
    m = folium.Map(location=[38.028028, -78.508516], zoom_start=14)

    # Define colors for clusters
    cluster_colors = {
        -1: "gray"  # Noise
    }
    unique_clusters = df["cluster"].unique()
    cmap = plt.cm.get_cmap("tab10", len(unique_clusters))

    for i, cluster in enumerate(unique_clusters):
        if cluster not in cluster_colors:
            cluster_colors[cluster] = f"#{''.join(np.random.choice(list('0123456789ABCDEF'), 6))}" 
    
    # adding points to a map
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,
            color=cluster_colors[row["cluster"]],
            fill=True,
            fill_color=cluster_colors[row["cluster"]],
            fill_opacity=0.6,
        ).add_to(m)

    return m

plot_clusters_folium(df_clusters)


C:\Users\chery\AppData\Local\Temp\ipykernel_38096\2389770946.py:12: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap("tab10", len(unique_clusters))


In [28]:
print(df_clusters['cluster'].nunique())

34


In [29]:
df_filtered = df_clusters[df_clusters["cluster"] != -1]
cluster_means = df_filtered.groupby("cluster")[["latitude", "longitude"]].mean().reset_index()
cluster_durations = df_filtered.groupby("cluster")["duration_time"].sum().reset_index()
cluster_means = cluster_means.merge(cluster_durations, on="cluster")
cluster_means.to_csv(cluster_data_file_path, index=False)

In [30]:
# plot ONLY the cluster means on the map
plot_clusters_folium(cluster_means)

C:\Users\chery\AppData\Local\Temp\ipykernel_38096\2389770946.py:12: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap("tab10", len(unique_clusters))


### Use Google Places API to find Location Names

In [31]:
import requests

# Define the API endpoint
url = "https://places.googleapis.com/v1/places:searchNearby"

# Define the headers
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": GOOGLE_API_KEY,  # Replace with your actual API key
    "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.types,places.id"
}

# Define the payload
data = {
    "maxResultCount": 1,
    "locationRestriction": {
        "circle": {
            "center": {
                "latitude": 38.904423,
                "longitude": -77.2036175000000
            },
            "radius": 20.0
        }
    }
}
# Make the POST request
response = requests.post(url, headers=headers, json=data)
print(response.json())

{'places': [{'id': 'ChIJHX0M4tdKtokRu7PPetQjRTY', 'types': ['grocery_store', 'market', 'liquor_store', 'supermarket', 'food_store', 'point_of_interest', 'food', 'store', 'health', 'establishment'], 'formattedAddress': '7514 Leesburg Pike, Falls Church, VA 22043, USA', 'displayName': {'text': "Trader Joe's", 'languageCode': 'en'}}]}


In [32]:
def get_place_info(lat, lon):
    url = "https://places.googleapis.com/v1/places:searchNearby"

    # Define the headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": GOOGLE_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.types,places.id"
    }

    # Define the payload
    data = {
        "maxResultCount": 1,
        "locationRestriction": {
            "circle": {
                "center": {
                    "latitude": lat,
                    "longitude": lon
                },
                "radius": 20.0
            }
        }
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()


In [33]:
cluster_means = pd.read_csv(cluster_data_file_path)
results = []

for index, row in cluster_means.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    time = row['duration_time']
    cluster_id = row['cluster']
    
    place_data = get_place_info(lat, lon)
    
    places_info = place_data.get('places', [])
    
    # Loop through the places and store the relevant info
    for place in places_info:
        results.append({
            'cluster_id': cluster_id,
            'latitude': lat,
            'longitude': lon,
            'total_duration_time': time,
            'place_id': place.get('id', ''),
            'place_types': place.get('types', []),
            'place_address': place.get('formattedAddress', ''),
            'place_name': place.get('displayName', {}).get('text', '')
        })


place_info_df = pd.DataFrame(results)
place_info_df.head()

,cluster_id,latitude,longitude,total_duration_time,place_id,place_types,place_address,place_name
0,0,38.028028,-78.508516,12 days 17:26:47.040000,ChIJwfbNe1OHs4kRs0PzrATWiPQ,"[apartment_complex, point_of_interest, establi...","1800 Jefferson Park Ave, Charlottesville, VA 2...",1800 JPA
1,1,38.031619,-78.510846,0 days 06:54:30.509000,ChIJB1s6kd6Hs4kRJhuNmO73ZLk,"[point_of_interest, establishment]","85 Engineer's Way, Charlottesville, VA 22903, USA",Rice Hall
2,2,38.052815,-78.501257,0 days 03:35:15.549000,ChIJI5T2nbeHs4kRuLewu_1gO8g,"[shopping_mall, point_of_interest, establishment]","1117 Emmet St N, Charlottesville, VA 22903, USA",Barracks Road Shopping Center
3,3,38.127095,-78.444288,0 days 00:58:41.853000,ChIJCf9IHQl3tIkR9yV1GWU27zw,"[shopping_mall, grocery_store, food_store, sto...","163 Community St, Charlottesville, VA 22911, USA",Hollymead Town Center
4,4,38.129001,-78.440783,0 days 00:39:53.183000,ChIJ_5xbqgh3tIkRC_j6tDv7bLs,"[department_store, shoe_store, child_care_agen...","312 Connor Dr, Charlottesville, VA 22911, USA",Target


In [34]:
place_info_df.to_csv(places_data_file_path)